In [17]:
injuries_df = pd.DataFrame(nfl.import_injuries([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009]))


In [1]:
#Import necessary libraries
import pandas as pd
import numpy as np
import nfl_data_py as nfl

In [39]:
#Download every available years worth of play by play, weekly, injury, and season information

#pbp_df contains play by play data which is aggregated to produce weekly player stats
pbp_df = pd.DataFrame(nfl.import_pbp_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

#weekly_df contains data from the import_weekly_data() endpoint and is used to grab the player's active team
weekly_df = pd.DataFrame(nfl.import_weekly_data([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

#injuries_df contains weekly injury data which will be merged with each position df to provide a count of positions that are injured on each side of the ball
injuries_df = pd.DataFrame(nfl.import_injuries([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009]))

#schedules_df contains defensive data used for strength metrics
schedules_df = pd.DataFrame(nfl.import_schedules([2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))


2024 done.
2023 done.
2022 done.
2021 done.
2020 done.
2019 done.
2018 done.
2017 done.
2016 done.
2015 done.
2014 done.
2013 done.
2012 done.
2011 done.
2010 done.
2009 done.
2008 done.
2007 done.
2006 done.
2005 done.
2004 done.
2003 done.
2002 done.
2001 done.
2000 done.
1999 done.
Downcasting floats.
Downcasting floats.


In [3]:
#Cell for testing
#Download every available years worth of play by play, weekly, injury, and season information

#pbp_df contains play by play data which is aggregated to produce weekly player stats
pbp_df = pd.DataFrame(nfl.import_pbp_data([2024]))#,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

#weekly_df contains data from the import_weekly_data() endpoint and is used to grab the player's active team
weekly_df = pd.DataFrame(nfl.import_weekly_data([2024]))#,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

#injuries_df contains weekly injury data which will be merged with each position df to provide a count of positions that are injured on each side of the ball
injuries_df = pd.DataFrame(nfl.import_injuries([2024]))#,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))

#schedules_df contains defensive data used for strength metrics
schedules_df = pd.DataFrame(nfl.import_schedules([2024]))#,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999]))


2024 done.
Downcasting floats.
Downcasting floats.


In [4]:
injuries_df.head()

,season,game_type,team,week,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
0,2024,REG,ARI,1,00-0035629,T,Jonah Williams,Jonah,Williams,Knee,None,Out,None,None,\n,2024-09-08 18:35:57
1,2024,REG,ARI,1,00-0039521,WR,Xavier Weaver,Xavier,Weaver,Oblique,None,Out,Oblique,None,Did Not Participate In Practice,2024-09-06 19:05:30
2,2024,REG,ARI,1,00-0037141,S,Joey Blount,Joey,Blount,Back,None,Questionable,Back,None,Limited Participation in Practice,2024-09-06 19:05:21
3,2024,REG,ARI,1,00-0039737,TE,Tip Reiman,Tip,Reiman,Ankle,None,Questionable,Ankle,None,Limited Participation in Practice,2024-09-06 19:05:21
4,2024,REG,ATL,1,00-0031583,DT,Grady Jarrett,Grady,Jarrett,None,None,None,Not injury related - resting player,None,Did Not Participate In Practice,2024-09-06 18:35:04


In [23]:
#Checking the columns in play by play
weekly_df.columns

Index(['player_id', 'player_name', 'player_display_name', 'position',
       'position_group', 'headshot_url', 'recent_team', 'season', 'week',
       'season_type', 'opponent_team', 'completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points

In [40]:
#Here, we aggregate each metric and group the result on the most relevant dimensions in the data set, although this list is not exhaustive and other dimensions may be added
passing_stats = pbp_df[['season', 'game_id', 'game_date', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'passer_player_id', 'passer_player_name']]
passing_stats = passing_stats[passing_stats['passer_player_id'].notnull()]
passing_stats = passing_stats.drop_duplicates(subset=['game_id', 'passer_player_id'])
passing_stats.head(50)

receiving_stats = pbp_df[['season','game_id', 'game_date', 'week', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'receiver_player_id', 'receiver_player_name']]
receiving_stats = receiving_stats[receiving_stats['receiver_player_id'].notnull()]
receiving_stats = receiving_stats.drop_duplicates(subset=['game_id', 'receiver_player_id'])
receiving_stats.head(50)

rushing_stats = pbp_df[['season','game_id', 'game_date', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'rusher_player_id', 'rusher_player_name']]
rushing_stats = rushing_stats[rushing_stats['rusher_player_id'].notnull()]
rushing_stats = rushing_stats.drop_duplicates(subset=['game_id', 'rusher_player_id'])
rushing_stats.head(50)


# passing_stats = pbp_df.groupby(['game_id', 'game_date', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'passer_player_id', 'passer_player_name']).agg({
#     'pass_attempt': 'sum',
#     'complete_pass': 'sum',
#     'passing_yards': 'sum',
#     'air_yards': 'sum',
#     'pass_touchdown': 'sum',
#     'interception': 'sum',
#     'rush_attempt': 'sum',
#     'rushing_yards': 'sum',# Sum passing yards
#     'rush_touchdown': 'sum',
#     'lateral_rush': 'sum',
#     'fumble': 'sum'
# }).reset_index()

#Checking the passing stats dataframe
# passing_stats.head(15)

,season,game_id,game_date,week,div_game,home_team,away_team,weather,stadium,spread_line,total_line,roof,surface,temp,wind,home_coach,away_coach,rusher_player_id,rusher_player_name
2,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0033553,J.Conner
11,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0035228,K.Murray
17,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0037248,J.Cook
46,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0039921,T.Benson
61,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0035537,Ty.Johnson
67,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0034857,J.Allen
84,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0039875,R.Davis
111,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0035500,G.Dortch
165,2024,2024_01_BAL_KC,2024-09-05,1,0,KC,BAL,"Temp: 67° F, Humidity: 53%, Wind: NE 8 mph",GEHA Field at Arrowhead Stadium,3.0,46.0,outdoors,grass,67.0,8.0,Andy Reid,John Harbaugh,00-0032764,D.Henry
175,2024,2024_01_BAL_KC,2024-09-05,1,0,KC,BAL,"Temp: 67° F, Humidity: 53%, Wind: NE 8 mph",GEHA Field at Arrowhead Stadium,3.0,46.0,outdoors,grass,67.0,8.0,Andy Reid,John Harbaugh,00-0034796,L.Jackson


In [27]:
receiving_stats = pbp_df[['season','game_id', 'game_date', 'week', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'receiver_player_id', 'receiver_player_name']]
receiving_stats = receiving_stats[receiving_stats['receiver_player_id'].notnull()]
receiving_stats = receiving_stats.drop_duplicates(subset=['game_id', 'receiver_player_id'])
receiving_stats.head(50)

,season,game_id,game_date,week,home_team,away_team,weather,stadium,spread_line,total_line,roof,surface,temp,wind,home_coach,away_coach,receiver_player_id,receiver_player_name
3,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0033553,J.Conner
8,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0035500,G.Dortch
9,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0039849,M.Harrison
14,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0038559,Mi.Wilson
19,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0033555,M.Hollins
23,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0038705,E.Demercado
24,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0037744,T.McBride
25,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0039041,E.Higgins
36,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0035689,D.Knox
45,2024,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0039921,T.Benson


In [ ]:
#Writing the dataframe to csv
#passing_stats.to_csv('nfl_passing_stats_1999_2024.csv')

In [6]:
#and we can do the same for rushing, receiving:

rushing_stats = pbp_df.groupby(['game_id', 'game_date', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'rusher_player_id', 'rusher_player_name']).agg({
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum', 
    'pass_attempt': 'sum',
    'complete_pass': 'sum',
    'interception': 'sum',
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'receiving_yards': 'sum',
    'yards_after_catch': 'sum',
    'fumble': 'sum'
}).reset_index()

rushing_stats.head()


,game_id,game_date,week,div_game,home_team,away_team,weather,stadium,spread_line,total_line,...,pass_attempt,complete_pass,interception,rush_attempt,rushing_yards,rush_touchdown,lateral_rush,receiving_yards,yards_after_catch,fumble
0,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,...,0.0,0.0,0.0,17.0,50.0,1.0,0.0,0.0,0.0,0.0
1,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,...,0.0,0.0,0.0,9.0,39.0,2.0,0.0,0.0,0.0,0.0
2,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,...,0.0,0.0,0.0,5.0,57.0,0.0,0.0,0.0,0.0,0.0
3,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,...,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0
4,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,...,0.0,0.0,0.0,2.0,7.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#rushing_stats.to_csv('nfl_rushing_stats_1999_2024.csv')

In [7]:
receiver_stats = pbp_df.groupby(['game_id', 'game_date', 'week', 'home_team', 'away_team', 'weather', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach', 'receiver_player_id', 'receiver_player_name']).agg({
    'passing_yards': 'sum',
    'air_yards': 'sum',
    'pass_touchdown': 'sum', 
    'pass_attempt': 'sum',
    'complete_pass': 'sum',
    'interception': 'sum', #the passing stats are duplicated for receivers
    'rush_attempt': 'sum',
    'rushing_yards': 'sum',# Sum passing yards
    'rush_touchdown': 'sum',
    'lateral_rush': 'sum',
    'receiving_yards': 'sum',
    'yards_after_catch':'sum',
    'touchdown':'sum',
    'receiver_id': 'count',
    'fumble': 'sum'
}).reset_index()


receiver_stats.head()

,game_id,game_date,week,home_team,away_team,weather,stadium,spread_line,total_line,roof,...,interception,rush_attempt,rushing_yards,rush_touchdown,lateral_rush,receiving_yards,yards_after_catch,touchdown,receiver_id,fumble
0,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,...,0.0,0.0,0.0,0.0,0.0,15.0,12.0,0.0,2,0.0
1,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,...,0.0,0.0,0.0,0.0,0.0,33.0,37.0,0.0,4,0.0
2,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,...,0.0,0.0,0.0,0.0,0.0,25.0,1.0,1.0,2,0.0
3,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,...,0.0,0.0,0.0,0.0,0.0,19.0,5.0,0.0,2,0.0
4,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,...,0.0,0.0,0.0,0.0,0.0,47.0,18.0,0.0,8,0.0


In [ ]:
#receiver_stats.to_csv('nfl_receiver_stats_1999_2024.csv')

In [21]:
#kicking isn't as straightforward since the data doesn't have the same data points as are available with passing, rushing, and receiving:
kicker_plays = pbp_df[pbp_df['kicker_player_name'].notnull()]

# Filter out only successful field goals for longest field goal calculation
made_field_goals = kicker_plays[kicker_plays['field_goal_result'] == 'made']

# Aggregate kicker statistics including longest field goal and kickoff
kicker_stats = kicker_plays.groupby(['season', 'game_id', 'game_date', 'week', 'div_game', 'home_team', 'away_team', 'weather', 'location', 'stadium',  'spread_line', 'total_line', 'roof', 'surface', 'temp', 'wind', 'home_coach', 'away_coach','kicker_player_id', 'kicker_player_name']).agg({
    'field_goal_result': lambda x: (x == 'made').sum(),  # Sum of successful field goals
    'extra_point_result': lambda x: (x == 'good').sum(),  # Sum of successful extra points
    'kick_distance': 'sum',  # Sum of kickoff yardage
    'field_goal_attempt': 'sum',  # Total field goal attempts
    'kickoff_attempt': 'sum'
}).reset_index()

# Calculate longest successful field goal using the filtered made_field_goals DataFrame
longest_fg = made_field_goals.groupby(['game_id','kicker_player_id', 'kicker_player_name'])['kick_distance'].max().reset_index()
longest_fg.rename(columns={'kick_distance': 'longest_fg'}, inplace=True)
# Merge longest field goal into kicker_stats
kicker_stats = pd.merge(kicker_stats, longest_fg, on=['game_id', 'kicker_player_id', 'kicker_player_name'], how='left')

# Rename the aggregated columns for clarity
kicker_stats.rename(columns={
    'field_goal_attempt': 'fg_attempts',
    'field_goal_result': 'fg_made',
    'longest_fg': 'longest_fg',
    'extra_point_result': 'xp_made',
    'kick_distance': 'total_kick_distance'
}, inplace=True)

kicker_stats.head()

kicker_stats.to_csv('nfl_kicker_stats_1999_2024.csv')

In [41]:
#Aggregating defensive performance for home and away teams
home_points = schedules_df[['game_id', 'season', 'week', 'home_team', 'away_score']].rename(
    columns={'home_team': 'team', 'away_score': 'points_allowed'}
)
home_points['adj_week'] = home_points['week']+1
home_points.head()

away_points = schedules_df[['game_id', 'season', 'week', 'away_team', 'home_score']].rename(
    columns={'away_team': 'team', 'home_score': 'points_allowed'}
)
away_points['adj_week'] = away_points['week']+1
away_points.head()

,game_id,season,week,team,points_allowed,adj_week
0,1999_01_MIN_ATL,1999,1,MIN,14.0,2
1,1999_01_KC_CHI,1999,1,KC,20.0,2
2,1999_01_PIT_CLE,1999,1,PIT,0.0,2
3,1999_01_OAK_GB,1999,1,OAK,28.0,2
4,1999_01_BUF_IND,1999,1,BUF,31.0,2


In [43]:
# Combine both home and dataframes
points_allowed_df = pd.concat([home_points, away_points], ignore_index=True)

# Sort by season, team, and week to calculate cumulative points allowed up to each week
points_allowed_df = points_allowed_df.sort_values(by=['season', 'team', 'week', 'adj_week'])

# Calculate cumulative points allowed for each team
points_allowed_df['cumulative_points_allowed'] = points_allowed_df.groupby(['season', 'team'])['points_allowed'].cumsum()

points_allowed_df.rename(columns={
    'team': 'defense_name'
}, inplace=True)

#Creating defensive ranking based on cumulative points
points_allowed_df['defensive_rank'] = points_allowed_df.groupby(['season', 'week'])['cumulative_points_allowed'].rank(method='min', ascending=True)

# Display the ranked defensive strength for verification
points_allowed_df.head(10)

,game_id,season,week,defense_name,points_allowed,adj_week,cumulative_points_allowed,defensive_rank
6986,1999_01_ARI_PHI,1999,1,ARI,24.0,2,24.0,15.0
7000,1999_02_ARI_MIA,1999,2,ARI,19.0,3,43.0,16.0
43,1999_03_SF_ARI,1999,3,ARI,24.0,4,67.0,17.0
7026,1999_04_ARI_DAL,1999,4,ARI,35.0,5,102.0,25.0
58,1999_05_NYG_ARI,1999,5,ARI,3.0,6,105.0,22.0
72,1999_06_WAS_ARI,1999,6,ARI,24.0,7,129.0,24.0
100,1999_08_NE_ARI,1999,8,ARI,27.0,9,156.0,20.0
7100,1999_09_ARI_NYJ,1999,9,ARI,12.0,10,168.0,17.0
128,1999_10_DET_ARI,1999,10,ARI,19.0,11,187.0,19.0
143,1999_11_DAL_ARI,1999,11,ARI,9.0,12,196.0,15.0


In [26]:
receiver_stats.head()

,game_id,game_date,week,home_team,away_team,weather,stadium,spread_line,total_line,roof,surface,temp,wind,home_coach,away_coach,receiver_player_id,receiver_player_name
3,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0033553,J.Conner
8,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0035500,G.Dortch
9,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0039849,M.Harrison
14,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0038559,Mi.Wilson
19,2024_01_ARI_BUF,2024-09-08,1,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Highmark Stadium,6.5,46.0,outdoors,a_turf,61.0,20.0,Sean McDermott,Jonathan Gannon,00-0033555,M.Hollins


In [44]:
#recent_team_df = weekly_df[['week', 'player_id', 'recent_team', 'completions', 'attempts','passing_yards', 'passing_tds', 'interceptions', 'sacks','carries','rushing_yards', 'rushing_tds', 'rushing_fumbles', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'target_share', 'air_yards_share','wopr']]

recent_team_df = weekly_df.groupby(['season', 'week', 'player_id', 'recent_team']).agg({
    'completions': 'sum', 
    'attempts': 'sum',
    'passing_yards': 'sum', 
    'passing_tds': 'sum', 
    'interceptions': 'sum', 
    'sacks': 'sum',
    'carries': 'sum',
    'rushing_yards': 'sum', 
    'rushing_tds': 'sum', 
    'rushing_fumbles': 'sum', 
    'receptions': 'sum', 
    'targets': 'sum', 
    'receiving_yards': 'sum', 
    'receiving_tds': 'sum'}).reset_index()

#Update the receivers dataframe with recent team data to be used in future joins
receivers_v0 = pd.merge(receiving_stats, recent_team_df, how = 'left', left_on = ['season', 'week', 'receiver_player_id'], right_on = ['season', 'week', 'player_id'])
receivers_v0.head()

#Update the rushers dataframe with recent team data to be used in future joins
rushers_v0 = pd.merge(rushing_stats, recent_team_df, how = 'left', left_on = ['season', 'week', 'rusher_player_id'], right_on = ['season', 'week', 'player_id'])
rushers_v0.head()

#Update the passers dataframe with recent team data to be used in future joins
passers_v0 = pd.merge(passing_stats, recent_team_df, how = 'left', left_on = ['season', 'week', 'passer_player_id'], right_on = ['season', 'week', 'player_id'])
passers_v0.head()


,season,game_id,game_date,week,div_game,home_team,away_team,weather,location,stadium,...,interceptions,sacks,carries,rushing_yards,rushing_tds,rushing_fumbles,receptions,targets,receiving_yards,receiving_tds
0,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Home,Highmark Stadium,...,0.0,4.0,5.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Home,Highmark Stadium,...,0.0,2.0,9.0,39.0,2.0,0.0,0.0,0.0,0.0,0.0
2,2024,2024_01_BAL_KC,2024-09-05,1,0,KC,BAL,"Temp: 67° F, Humidity: 53%, Wind: NE 8 mph",Home,GEHA Field at Arrowhead Stadium,...,0.0,1.0,16.0,122.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024,2024_01_BAL_KC,2024-09-05,1,0,KC,BAL,"Temp: 67° F, Humidity: 53%, Wind: NE 8 mph",Home,GEHA Field at Arrowhead Stadium,...,1.0,2.0,2.0,3.0,0.0,0.0,1.0,1.0,2.0,0.0
4,2024,2024_01_CAR_NO,2024-09-08,1,1,NO,CAR,"Cloudy Temp: 75° F, Humidity: 64%, Wind: C 13 mph",Home,Caesars Superdome,...,0.0,1.0,2.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
#Adding home and away field for defensive stats join
receivers_v0['home_away'] = receivers_v0.apply(
    lambda row: 'Home' if row['recent_team'] == row['home_team'] else 'Away', axis=1
)
receivers_v0.head()

rushers_v0['home_away'] = rushers_v0.apply(
    lambda row: 'Home' if row['recent_team'] == row['home_team'] else 'Away', axis=1
)
rushers_v0.head()

passers_v0['home_away'] = passers_v0.apply(
    lambda row: 'Home' if row['recent_team'] == row['home_team'] else 'Away', axis=1
)
passers_v0.head()

# kickers_v0['home_away'] = kickers_v0.apply(
#     lambda row: 'Home' if row['recent_team'] == row['home_team'] else 'Away', axis=1
# )
# kickers_v0.head()

,season,game_id,game_date,week,div_game,home_team,away_team,weather,location,stadium,...,sacks,carries,rushing_yards,rushing_tds,rushing_fumbles,receptions,targets,receiving_yards,receiving_tds,home_away
0,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Home,Highmark Stadium,...,4.0,5.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,Away
1,2024,2024_01_ARI_BUF,2024-09-08,1,0,BUF,ARI,"Clouds and sun with wind Temp: 61° F, Humidity...",Home,Highmark Stadium,...,2.0,9.0,39.0,2.0,0.0,0.0,0.0,0.0,0.0,Home
2,2024,2024_01_BAL_KC,2024-09-05,1,0,KC,BAL,"Temp: 67° F, Humidity: 53%, Wind: NE 8 mph",Home,GEHA Field at Arrowhead Stadium,...,1.0,16.0,122.0,0.0,0.0,0.0,0.0,0.0,0.0,Away
3,2024,2024_01_BAL_KC,2024-09-05,1,0,KC,BAL,"Temp: 67° F, Humidity: 53%, Wind: NE 8 mph",Home,GEHA Field at Arrowhead Stadium,...,2.0,2.0,3.0,0.0,0.0,1.0,1.0,2.0,0.0,Home
4,2024,2024_01_CAR_NO,2024-09-08,1,1,NO,CAR,"Cloudy Temp: 75° F, Humidity: 64%, Wind: C 13 mph",Home,Caesars Superdome,...,1.0,2.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,Home


In [46]:
#adding defensive stats to receiver dataframe
home_team_receivers = receivers_v0[receivers_v0['home_away'] == 'Home']
home_team_receivers_w_defense = pd.merge(home_team_receivers, points_allowed_df, how = 'left', left_on = [ 'season', 'away_team', 'week'], right_on = ['season','defense_name', 'adj_week'])
home_team_receivers_w_defense[home_team_receivers_w_defense['week_x'] == 2].head()


away_team_receivers = receivers_v0[receivers_v0['home_away'] == 'Away']
away_team_receivers_w_defense = pd.merge(away_team_receivers, points_allowed_df, how = 'left', left_on = [ 'season','home_team', 'week'], right_on = ['season','defense_name', 'adj_week'])

away_team_receivers_w_defense[away_team_receivers_w_defense['week_x'] == 2].head()

#adding defensive stats to rusher dataframe
home_team_rushers = rushers_v0[rushers_v0['home_away'] == 'Home']
home_team_rushers_w_defense = pd.merge(home_team_rushers, points_allowed_df, how = 'left', left_on = [ 'season','away_team', 'week'], right_on = ['season','defense_name', 'adj_week'])
home_team_rushers_w_defense[home_team_rushers_w_defense['week_x'] == 2].head()

away_team_rushers = rushers_v0[rushers_v0['home_away'] == 'Away']
away_team_rushers_w_defense = pd.merge(away_team_rushers, points_allowed_df, how = 'left', left_on = [ 'season','home_team', 'week'], right_on = ['season','defense_name', 'adj_week'])

away_team_rushers_w_defense[away_team_rushers_w_defense['week_x'] == 2].head()

#adding defensive stats to passer dataframe
home_team_passers = passers_v0[passers_v0['home_away'] == 'Home']
home_team_passers_w_defense = pd.merge(home_team_passers, points_allowed_df, how = 'left', left_on = [ 'season','away_team', 'week'], right_on = ['season','defense_name', 'adj_week'])
home_team_passers_w_defense[home_team_passers_w_defense['week_x'] == 2].head()

away_team_passers = passers_v0[passers_v0['home_away'] == 'Away']
away_team_passers_w_defense = pd.merge(away_team_passers, points_allowed_df, how = 'left', left_on = [ 'season','home_team', 'week'], right_on = ['season','defense_name', 'adj_week'])

away_team_passers_w_defense[away_team_passers_w_defense['week_x'] == 2].head()


#adding defensive stats to kicker dataframe
# home_team_kickers = kickers_v0[kickers_v0['home_away'] == 'Home']
# home_team_kickers_w_defense = pd.merge(home_team_kickers, points_allowed_df, how = 'left', left_on = [ 'away_team', 'week'], right_on = ['defense_name', 'adj_week'])
# home_team_kickers_w_defense[home_team_kickers_w_defense['week_x'] == 2].head()

# away_team_kickers = kickers_v0[kickers_v0['home_away'] == 'Away']
# away_team_kickers_w_defense = pd.merge(away_team_kickers, points_allowed_df, how = 'left', left_on = [ 'home_team', 'week'], right_on = ['defense_name', 'adj_week'])

# away_team_kickers_w_defense[away_team_kickers_w_defense['week_x'] == 2].head()




,season,game_id_x,game_date,week_x,div_game,home_team,away_team,weather,location,stadium,...,receiving_yards,receiving_tds,home_away,game_id_y,week_y,defense_name,points_allowed,adj_week,cumulative_points_allowed,defensive_rank
19,2024,2024_02_ATL_PHI,2024-09-16,2,0,PHI,ATL,"Cloudy Temp: 71° F, Humidity: 72%, Wind: E 9 mph",Home,Lincoln Financial Field,...,0.0,0.0,Away,2024_01_GB_PHI,1.0,PHI,29.0,2.0,29.0,25.0
20,2024,2024_02_BUF_MIA,2024-09-12,2,1,MIA,BUF,"Partly Cloudy Temp: 86° F, Humidity: 79%, Wind...",Home,Hard Rock Stadium,...,0.0,0.0,Away,2024_01_JAX_MIA,1.0,MIA,17.0,2.0,17.0,7.0
21,2024,2024_02_CHI_HOU,2024-09-15,2,0,HOU,CHI,"Clear Temp: 89° F, Humidity: 57%, Wind: SSW 4 mph",Home,NRG Stadium,...,0.0,0.0,Away,2024_01_HOU_IND,1.0,HOU,27.0,2.0,27.0,21.0
22,2024,2024_02_CIN_KC,2024-09-15,2,0,KC,CIN,"Temp: 89° F, Humidity: 44%, Wind: SE 8 mph",Home,GEHA Field at Arrowhead Stadium,...,0.0,0.0,Away,2024_01_BAL_KC,1.0,KC,20.0,2.0,20.0,12.0
23,2024,2024_02_CLE_JAX,2024-09-15,2,0,JAX,CLE,"Temp: 80° F, Humidity: 78%, Wind: N 12 mph",Home,EverBank Stadium,...,0.0,0.0,Away,2024_01_JAX_MIA,1.0,JAX,20.0,2.0,20.0,12.0


In [47]:
receivers_full = pd.concat([home_team_receivers_w_defense, away_team_receivers_w_defense], ignore_index=True)
rushers_full = pd.concat([home_team_rushers_w_defense, away_team_rushers_w_defense], ignore_index=True)
passers_full = pd.concat([home_team_passers_w_defense, away_team_passers_w_defense], ignore_index=True)
# kickers_full = pd.concat([home_team_kickers_w_defense, away_team_kickers_w_defense], ignore_index=True)


In [48]:
#Grabbing positions from the defense_df and aggregating counts of injured players on either team
unique_positions = injuries_df['position'].unique()
unique_statuses = injuries_df['report_status'].unique()
unique_positions, unique_statuses

# Define relevant offensive and defensive positions for flag creation
offensive_positions = ['WR', 'RB', 'QB', 'TE', 'T', 'G', 'C']
defensive_positions = ['S', 'DT', 'DE', 'CB', 'LB']

# Create flags for offensive injuries
for position in offensive_positions:
    injuries_df[f'offense_injury_{position}'] = (
        (injuries_df['position'] == position) & 
        (injuries_df['report_status'].isin(['Out', 'Questionable']))
    ).astype(int)

# Create flags for defensive injuries
for position in defensive_positions:
    injuries_df[f'defense_injury_{position}'] = (
        (injuries_df['position'] == position) & 
        (injuries_df['report_status'].isin(['Out', 'Questionable']))
    ).astype(int)

# Aggregate the flags by team and week to count injuries in each category
injuries_df_v0 = injuries_df.groupby(['season', 'team', 'week']).agg({
    f'offense_injury_{position}': 'sum' for position in offensive_positions
}).join(
    injuries_df.groupby(['season','team', 'week']).agg({
        f'defense_injury_{position}': 'sum' for position in defensive_positions
    })
).reset_index()

# Display the resulting injury flags data
injuries_df_v0.head()

,season,team,week,offense_injury_WR,offense_injury_RB,offense_injury_QB,offense_injury_TE,offense_injury_T,offense_injury_G,offense_injury_C,defense_injury_S,defense_injury_DT,defense_injury_DE,defense_injury_CB,defense_injury_LB
0,2009.0,ARI,1.0,2,0,1,0,0,0,0,0,0,0,0,0
1,2009.0,ARI,2.0,3,0,0,0,0,0,0,1,0,1,0,1
2,2009.0,ARI,3.0,1,0,0,0,1,0,0,2,0,1,0,2
3,2009.0,ARI,5.0,1,0,0,0,0,1,0,0,1,1,1,0
4,2009.0,ARI,6.0,1,1,0,1,0,0,0,0,0,1,1,1


In [49]:
defensive_inj = injuries_df_v0[['season', 'team', 'week', 'defense_injury_S', 'defense_injury_DT', 'defense_injury_DE', 'defense_injury_CB', 'defense_injury_LB']]
offensive_inj = injuries_df_v0[['season', 'team', 'week', 'offense_injury_WR', 'offense_injury_RB', 'offense_injury_QB', 'offense_injury_TE', 'offense_injury_T', 'offense_injury_G', 'offense_injury_C']]

defensive_inj.rename(columns={
    'team': 'defensive_team',
    'week': 'defense_week'
}, inplace=True)

offensive_inj.rename(columns={
    'team': 'offensive_team',
    'week': 'offense_week'
}, inplace=True)

C:\Users\thela\AppData\Local\Temp\ipykernel_9496\3024446815.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  defensive_inj.rename(columns={
C:\Users\thela\AppData\Local\Temp\ipykernel_9496\3024446815.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offensive_inj.rename(columns={


In [50]:
receiver_final = receivers_full[receivers_full['home_away']=='Away'].merge(defensive_inj, how = 'left', left_on=['season','home_team', 'week_x'], right_on=['season','defensive_team', 'defense_week']).merge(offensive_inj, how = 'left', left_on=['season','away_team', 'week_x'], right_on=['season','offensive_team', 'offense_week'])
receiver_final = receivers_full[receivers_full['home_away']=='Home'].merge(defensive_inj, how = 'left', left_on=['season','away_team', 'week_x'], right_on=['season','defensive_team', 'defense_week']).merge(offensive_inj, how = 'left', left_on=['season','home_team', 'week_x'], right_on=['season','offensive_team', 'offense_week'])

receiver_final.to_csv('nfl_receiver_stats_1999_2024_20241029.csv')

rusher_final = rushers_full[rushers_full['home_away']=='Away'].merge(defensive_inj, how = 'left', left_on=['season','home_team', 'week_x'], right_on=['season','defensive_team', 'defense_week']).merge(offensive_inj, how = 'left', left_on=['season','away_team', 'week_x'], right_on=['season','offensive_team', 'offense_week'])
rusher_final = rushers_full[rushers_full['home_away']=='Home'].merge(defensive_inj, how = 'left', left_on=['season','away_team', 'week_x'], right_on=['season','defensive_team', 'defense_week']).merge(offensive_inj, how = 'left', left_on=['season','home_team', 'week_x'], right_on=['season','offensive_team', 'offense_week'])

rusher_final.to_csv('nfl_rusher_stats_1999_2024_20241029.csv')
# rusher_final.head(50).to_csv('sample_nfl_rusher_stats_1999_2024_20241029.csv')

passer_final = passers_full[passers_full['home_away']=='Away'].merge(defensive_inj, how = 'left', left_on=['season','home_team', 'week_x'], right_on=['season','defensive_team', 'defense_week']).merge(offensive_inj, how = 'left', left_on=['season','away_team', 'week_x'], right_on=['season','offensive_team', 'offense_week'])
passer_final = passers_full[passers_full['home_away']=='Home'].merge(defensive_inj, how = 'left', left_on=['season','away_team', 'week_x'], right_on=['season','defensive_team', 'defense_week']).merge(offensive_inj, how = 'left', left_on=['season','home_team', 'week_x'], right_on=['season','offensive_team', 'offense_week'])

passer_final.to_csv('nfl_passer_stats_1999_2024_20241029.csv')




In [ ]:
receiver_final.to_csv('nfl_receiver_stats_1999_2024_20241029.csv')
rusher_final.to_csv('nfl_rusher_stats_1999_2024_20241029.csv')
passer_final.to_csv('nfl_passer_stats_1999_2024_20241029.csv')

In [38]:
receiver_final.head(50).to_csv('sample_nfl_receiver_stats_1999_2024_20241029.csv')
rusher_final.head(50).to_csv('sample_nfl_rusher_stats_1999_2024_20241029.csv')
passer_final.head(50).to_csv('sample_nfl_passer_stats_1999_2024_20241029.csv')